</h1></center><center><h1 style="font-size:280%; font-family:cursive; background:yellow; color:black; border-radius:10px 10px; padding:10px;">Movie Recommendation System Project using ML
</h1></center>

<p style="font-size:150%; font-family:cursive;">The main goal of this machine learning project is to build a recommendation engine that recommends movies to users. This Python project is designed to help you understand the functioning of how a recommendation system works. We will be developing an Item Based Collaborative Filter. By the end of this tutorial, you will gain experience of implementing your Python, Data Science, and Machine learning skills in a real-life project.
Before moving ahead in this movie recommendation system project in ML, you need to know what recommendation system means.</p>

<center><h1 style="font-size:280%; font-family:cursive; background:yellow; color:black; border-radius:10px 10px; padding:10px;">What is a Recommendation System?
</h1></center>


<p style="font-size:150%; font-family:cursive;">A recommendation system provides suggestions to the users through a filtering process that is based on user preferences and browsing history. The information about the user is taken as an input. The information is taken from the input that is in the form of browsing data. This information reflects the prior usage of the product as well as the assigned ratings. A recommendation system is a platform that provides its users with various contents based on their preferences and likings. A recommendation system takes the information about the user as an input. The recommendation system is an implementation of the machine learning algorithms.</p>

<center><img src="https://developer-blogs.nvidia.com/wp-content/uploads/2021/04/Whats-recommendation-system_Pic-2.png"></center>

<p style="font-size:150%; font-family:cursive;">A recommendation system also finds a similarity between the different products. For example, Netflix Recommendation System provides you with the recommendations of the movies that are similar to the ones that have been watched in the past. Furthermore, there is a collaborative content filtering that provides you with the recommendations in respect with the other users who might have a similar viewing history or preferences. There are two types of recommendation systems – Content-Based Recommendation System and Collaborative Filtering Recommendation. In this project of recommendation system in R, we will work on a collaborative filtering recommendation system and more specifically, ITEM based collaborative recommendation system.</p>

</h1></center><center><h1 style="font-size:180%; font-family:cursive; background:pink; color:black; border-radius:10px 10px; padding:10px;"> Step 1. Work with IMBD Movie Dataset
</h1></center>

In [ ]:
# import necessary Libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
movies_meta=pd.read_csv("../input/imdb-5000-movie-dataset/movie_metadata.csv")

In [ ]:
movies_meta.head()

In [ ]:
movies_meta.shape

In [ ]:
movies_meta.columns

In [ ]:
# Plot a graph which showing the no of movies release in a specific year
movies_meta.title_year.value_counts(dropna=False).sort_index().plot(kind="barh",figsize=(15,16))
plt.show()

In [ ]:
# Select these columns for data Preprocessing
movies_meta=movies_meta.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [ ]:
movies_meta.head()

In [ ]:
# Replace Nan values to "unknown" in these columns
movies_meta['actor_1_name']=movies_meta['actor_1_name'].replace(np.nan,"unknown")
movies_meta['actor_2_name']=movies_meta['actor_2_name'].replace(np.nan,"unknown")
movies_meta['actor_3_name']=movies_meta['actor_3_name'].replace(np.nan,"unknown")
movies_meta['director_name']=movies_meta['director_name'].replace(np.nan,"unknown")

In [ ]:
# Split "genres" column values by " "
movies_meta['genres'] = movies_meta['genres'].str.replace('|', ' ')

In [ ]:
# Lower the values of " movies_title " column
movies_meta['movie_title']=movies_meta['movie_title'].str.lower()

In [ ]:
# null terminating char at the end
movies_meta['movie_title'][1]

In [ ]:
# removing the null terminating char at the end
movies_meta['movie_title'] = movies_meta['movie_title'].apply(lambda x : x[:-1])

In [ ]:
movies_meta["movie_title"][1]

</h1></center><center><h1 style="font-size:180%; font-family:cursive; background:pink; color:black; border-radius:10px 10px; padding:10px;"> Step 2. Work with  Movies Credit and Meta Datasets
</h1></center>

In [ ]:
# Read Datasets
credit_data=pd.read_csv("../input/the-movies-dataset/credits.csv")

In [ ]:
credit_data.head()

In [ ]:
credit_data.info()

In [ ]:
# Read Meta dataset
meta=pd.read_csv("../input/the-movies-dataset/movies_metadata.csv")

In [ ]:
meta.head()

In [ ]:
# Extract years from " release_date "
meta["release_date"]=pd.to_datetime(meta["release_date"],errors="coerce")
meta['year'] = meta['release_date'].dt.year
meta["year"].value_counts().sort_index()

In [ ]:
# In meta dataset Extract 'genres','id','title','year' of 2017 movies
new_meta=meta.loc[meta.year==2017,['genres','id','title','year']]
new_meta.head()

In [ ]:
new_meta.info()

In [ ]:
new_meta.shape

In [ ]:
# Convert Id column datatype to int
new_meta["id"]=new_meta["id"].astype(int)


In [ ]:
# merge modified meta data and credit data on id column
data = pd.merge(new_meta, credit_data, on='id')
data.head()

In [ ]:
pd.set_option('display.max_colwidth', 75)


In [ ]:
data["genres"].dtypes

In [ ]:
import ast
data["genres"]=data["genres"].map(lambda x : ast.literal_eval(x))
data["cast"]=data["cast"].map(lambda x : ast.literal_eval(x))
data["crew"]=data["crew"].map(lambda x : ast.literal_eval(x))

In [ ]:
data["genres"].dtypes

In [ ]:
# Create a function which convert to a list of genres
def make_genresList(x):
    gen = []
    st = " "
    for i in x:
        if i.get('name') == 'Science Fiction':
            scifi = 'Sci-Fi'
            gen.append(scifi)
        else:
            gen.append(i.get('name'))
    if gen == []:
        return np.NaN
    else:
        return (st.join(gen))

In [ ]:
data['genres_list'] = data['genres'].map(lambda x: make_genresList(x))
data["genres_list"]

In [ ]:
# Create a function which extract first actor from "cast" column
def get_actor1(x):
    casts=[]
    for i in x:
        casts.append(i.get("name"))
        if casts==[]:
            return np.Nan
        else:
            return (casts[0])

In [ ]:
data['actor_1_name'] = data['cast'].map(lambda x: get_actor1(x))
data["actor_1_name"]

In [ ]:
# Create a function which extract second actor from "cast" column
def get_actor2(x):
    casts = []
    for i in x:
        casts.append(i.get('name'))
    if casts == [] or len(casts)<=1:
        return np.NaN
    else:
        return (casts[1])

In [ ]:
data['actor_2_name'] = data['cast'].map(lambda x: get_actor2(x))
data["actor_2_name"]

In [ ]:
# Create a function which extract third actor from "cast" column
def get_actor3(x):
    casts = []
    for i in x:
        casts.append(i.get('name'))
    if casts == [] or len(casts)<=2:
        return np.NaN
    else:
        return (casts[2])

In [ ]:
data['actor_3_name'] = data['cast'].map(lambda x: get_actor3(x))
data["actor_3_name"]

In [ ]:
# Create a function which extract Director name from "crew" column
def get_directors(x):
    dt = []
    st = " "
    for i in x:
        if i.get('job') == 'Director':
            dt.append(i.get('name'))
    if dt == []:
        return np.NaN
    else:
        return (st.join(dt))

In [ ]:
data['director_name'] = data['crew'].map(lambda x: get_directors(x))
data['director_name']

In [ ]:
data.columns

In [ ]:
# Choose only these columns for data preprocessing
movies_data=data.loc[:,['actor_1_name','actor_2_name', 'actor_3_name', 'director_name',"genres_list",'title']]
movies_data.head()

In [ ]:
# Check null values
movies_data.isna().sum()

In [ ]:
# Drop null values
movies_data=movies_data.dropna(how="any")
movies_data.isna().sum()

In [ ]:
# Rename columns
movies_data=movies_data.rename(columns={'genres_list':'genres'})
movies_data=movies_data.rename(columns={'title':'movie_title'})


In [ ]:
movies_data["movie_title"]=movies_data["movie_title"].str.lower()

In [ ]:
# Make a column which is combination of all the columns of dataset
movies_data["comb"]=movies_data["actor_1_name"]+' '+movies_data["actor_2_name"]+" "+ movies_data["actor_3_name"]+' ' + movies_data["director_name"]
movies_data.head()

In [ ]:
# Read old data which is stored in data.csv
old_data=pd.read_csv("../input/movie-recommendation-data/data.csv")
old_data.head()

In [ ]:
# Also for this Data Make a column which is combination of all the columns of dataset
old_data['comb'] = old_data['actor_1_name'] + ' ' + old_data['actor_2_name'] + ' '+ old_data['actor_3_name'] + ' '+ old_data['director_name'] +' ' + old_data['genres']
old_data.head()

In [ ]:
# Append  old and new data
new_data=old_data.append(movies_data)
new_data

In [ ]:
new_data.drop_duplicates(subset ="movie_title", keep = 'last', inplace = True)
new_data

In [ ]:
new_data.to_csv('new_data.csv',index=False)


</h1></center><center><h1 style="font-size:180%; font-family:cursive; background:pink; color:black; border-radius:10px 10px; padding:10px;"> Step 3. Work with 2018 Movies dataset
</h1></center>

In [ ]:
# Load data from wikipedia
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"
data1=pd.read_html(link,header=0)[2]
data2=pd.read_html(link,header=0)[3]
data3=pd.read_html(link,header=0)[4]
data4=pd.read_html(link,header=0)[5]

In [ ]:
# append all the data
all_data=data1.append(data2.append(data3.append(data4,ignore_index=True),ignore_index=True),ignore_index=True)

In [ ]:
all_data

In [ ]:
# install TMBDV3API for access of movies data 
!pip install tmdbv3api

In [ ]:
# Access your api keys from TMBD website
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key='26dd2b4303a353ce9da1643a477fbbf4'

In [ ]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    movie_id = result[0].id
    response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
    data_json = response.json()
    if data_json['genres']:
        genre_str = " " 
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return genre_str.join(genres)
    else:
        np.NaN

In [ ]:
all_data['genres'] = all_data['Title'].map(lambda x: get_genre(str(x)))

In [ ]:
all_data

In [ ]:
all_data=all_data[['Title','Cast and crew','genres']]
all_data

In [ ]:
# Create a function which extract Director name from "Cast and crew" column
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif  " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
         return x.split(" (director/screenplay)")[0]

In [ ]:
all_data["director_name"]=all_data['Cast and crew'].map(lambda x : get_director(x))

In [ ]:
all_data

In [ ]:
# Create a function which extract first actor from "cast and crew" column
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [ ]:
all_data['actor_1_name']=all_data["Cast and crew"].map(lambda x : get_actor1(x))
all_data

In [ ]:
# Create a function which extract second actor from "cast and crew" column
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [ ]:
all_data["actor_2_name"]=all_data["Cast and crew"].map(lambda x : get_actor2(x))

In [ ]:
all_data

In [ ]:
# Create a function which extract third actor from "cast and crew" column
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [ ]:
all_data["actor_3_name"]=all_data["Cast and crew"].map(lambda x : get_actor3(x))
all_data

In [ ]:
# Rename title column
all_data=all_data.rename(columns={'Title': 'movie_title'})

In [ ]:
# Select these columns for data preprocessing
all_data=all_data.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]
all_data

In [ ]:
all_data["actor_2_name"]=all_data["actor_2_name"].replace(np.nan,"unknown")
all_data["actor_3_name"]=all_data["actor_3_name"].replace(np.nan,"unknown")

In [ ]:
all_data["movie_title"]=all_data["movie_title"].str.lower()

In [ ]:
all_data

In [ ]:
all_data['comb'] = all_data['actor_1_name'] + ' ' + all_data['actor_2_name'] + ' '+ all_data['actor_3_name'] + ' '+ all_data['director_name'] +' ' + all_data['genres']

In [ ]:
all_data


</h1></center><center><h1 style="font-size:180%; font-family:cursive; background:pink; color:black; border-radius:10px 10px; padding:10px;"> Step 4. Work with 2019 Movies dataset
</h1></center>

In [ ]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [ ]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)
df

In [ ]:
df['genres'] = df['Title'].map(lambda x: get_genre(str(x)))

In [ ]:
df_2019 = df[['Title','Cast and crew','genres']]
df_2019

In [ ]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [ ]:
df_2019['director_name'] = df_2019['Cast and crew'].map(lambda x: get_director(str(x)))

In [ ]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [ ]:
df_2019['actor_1_name'] = df_2019['Cast and crew'].map(lambda x: get_actor1(x))

In [ ]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [ ]:
df_2019['actor_2_name'] = df_2019['Cast and crew'].map(lambda x: get_actor2(x))

In [ ]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [ ]:
df_2019['actor_3_name'] = df_2019['Cast and crew'].map(lambda x: get_actor3(x))

In [ ]:
df_2019 = df_2019.rename(columns={'Title':'movie_title'})

In [ ]:
new_df19 = df_2019.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [ ]:
new_df19['actor_2_name'] = new_df19['actor_2_name'].replace(np.nan, 'unknown')
new_df19['actor_3_name'] = new_df19['actor_3_name'].replace(np.nan, 'unknown')

In [ ]:
new_df19['movie_title'] = new_df19['movie_title'].str.lower()

In [ ]:
new_df19['comb'] = new_df19['actor_1_name'] + ' ' + new_df19['actor_2_name'] + ' '+ new_df19['actor_3_name'] + ' '+ new_df19['director_name'] +' ' + new_df19['genres']
new_df19

In [ ]:
my_df = all_data.append(new_df19,ignore_index=True)
my_df

In [ ]:
final_df =new_data.append(my_df,ignore_index=True)
final_df

In [ ]:
final_df.isna().sum()

In [ ]:
final_df = final_df.dropna(how='any')
final_df.isna().sum()


</h1></center><center><h1 style="font-size:180%; font-family:cursive; background:pink; color:black; border-radius:10px 10px; padding:10px;"> Step 5. Work with 2020 Movies dataset
</h1></center>

In [ ]:
import requests
import bs4 as bs
import urllib.request

In [ ]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [ ]:
tables = soup.find_all('table',class_='wikitable sortable')
len(tables)

In [ ]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0]

In [ ]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)
df

In [ ]:
df_2020 = df[['Title','Cast and crew']]
df_2020

In [ ]:
df_2020['genres'] = df_2020['Title'].map(lambda x: get_genre(str(x)))

In [ ]:
df_2020

In [ ]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [ ]:
df_2020['director_name'] = df_2020['Cast and crew'].map(lambda x: get_director(str(x)))

In [ ]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [ ]:
df_2020['actor_1_name'] = df_2020['Cast and crew'].map(lambda x: get_actor1(str(x)))

In [ ]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [ ]:
df_2020['actor_2_name'] = df_2020['Cast and crew'].map(lambda x: get_actor2(str(x)))

In [ ]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [ ]:
df_2020['actor_3_name'] = df_2020['Cast and crew'].map(lambda x: get_actor3(str(x)))
df_2020

In [ ]:
df_2020 = df_2020.rename(columns={'Title':'movie_title'})

In [ ]:
new_df20 = df_2020.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]
new_df20

In [ ]:
new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']

In [ ]:
new_df20 = new_df20.dropna(how='any')

In [ ]:
new_df20.isna().sum()

In [ ]:
new_df20['movie_title'] = new_df20['movie_title'].str.lower()
new_df20

In [ ]:
final_df = new_data.append(new_df20,ignore_index=True)
final_df


</h1></center><center><h1 style="font-size:180%; font-family:cursive; background:pink; color:black; border-radius:10px 10px; padding:10px;"> Step 6. Now work with sentiment Analysis of Movies Reviews
</h1></center>

In [ ]:
# import necessary libraries
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score,accuracy_score


In [ ]:
nltk.download("stopwords")

In [ ]:
# Load dataset of movie reviews
datasets=pd.read_csv('../input/movie-recommendation-data/reviews.txt',sep = '\t', names =['Reviews','Comments'])

In [ ]:
datasets.head()

In [ ]:
stopset=set(stopwords.words("english"))

In [ ]:
# apply tfidf vectorizer 
vectorizer=TfidfVectorizer(use_idf = True,lowercase = True, strip_accents='ascii',stop_words=stopset)

In [ ]:
x=vectorizer.fit_transform(datasets.Comments)
y=datasets.Reviews

In [ ]:
# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [ ]:
clf = naive_bayes.MultinomialNB()
clf.fit(X_train,y_train)

In [ ]:
# Check your score
accuracy_score(y_test,clf.predict(X_test))*100